In [114]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [115]:
data = pd.read_pickle('train_data.pkl')

In [116]:
data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'LapPercent_1', 'LapPosition_1',
       'Sector1Percent_1', 'Sector1Position_1', 'Sector2Percent_1',
       'Sector2Position_1', 'Sector3Percent_1', 'Sector3Position_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'LapPercent_1_1', 'LapPosition_1_1', 'Sector1Percent_1_1',
       'Sector1Position_1_1', 'Sector2Percent_1_1', 'Sector2Position_1_1',
       'Sector3Percent_1_1', 'Sector3Position_1_1', 'LapTime_2',
       'Sector1Time_2', 'Sector2Time_2', 'Sector3Time_2', 'LapPercent_2',
       'LapPosition_2', 'Sector1Percent_2', 'Sector1Position_2',
       'Sector2Percent_2', 'Sector2Position_2', 'Sector3Percent_2',
       'Sector3Position_2', 'LapTime_1_2', 'Sector1Time_1_2',
       'Sector2Time_1_2', 'Sector3Time_1_2', 'LapPercent_1_2',
       'LapPosition_1_2', 'Sector1Percent_1_2', 'Sector1Position_1_2',
       'Sector2Percent_1_2', '

In [117]:
data = data.dropna()

In [118]:
data.iloc[0]

DriverNumber                           10
Team                           AlphaTauri
Driver                                GAS
LapTime_1          0 days 00:01:19.062000
Sector1Time_1      0 days 00:00:22.362000
                            ...          
PrevPosition1                         9.0
PrevLapPercent2                  0.005093
PrevPosition2                         5.0
PrevLapPercent3                  0.009124
PrevPosition3                         5.0
Name: 60, Length: 66, dtype: object

In [119]:
train_data = data[data['EventDate'] <= np.datetime64('2022-06-30')]
test_data = data[data['EventDate'] > np.datetime64('2022-06-30')]

In [124]:
X_cols =['LapPosition_1', 'LapPosition_2', 'PrevPosition1',
       'PrevPosition2', 'PrevPosition3']

In [125]:
train_X = train_data[X_cols]
train_y = train_data['Position']

In [126]:
# for col in train_X.columns:
#     percentiles = train_X[col].quantile([0, 0.99]).values
#     train_X.loc[train_X[col] >= percentiles[1], col] = percentiles[1]
# for col in train_X.columns:
#     lower_percentile = 0.05
#     higher_percentile = 0.95

#     low, high = train_X[col].quantile([lower_percentile, higher_percentile])

#     #cap values above high to high
#     train_X.loc[:,col] = train_X.loc[:,col].apply(lambda x: high if x > high else x)#[train_X[col] > high] = high


In [127]:
train_X.to_pickle('train_X.pkl')

In [128]:
# train_X = (train_X - train_X.mean()) / train_X.std()

In [129]:
train_X.columns.size

5

In [130]:
train_X

,LapPosition_1,LapPosition_2,PrevPosition1,PrevPosition2,PrevPosition3
60,16.0,10.0,9.0,5.0,5.0
62,10.0,3.0,13.0,15.0,9.0
63,9.0,7.0,8.0,4.0,4.0
64,3.0,2.0,17.0,10.0,10.0
66,0.0,5.0,16.0,6.0,6.0
...,...,...,...,...,...
533,11.0,14.0,20.0,15.0,10.0
534,19.0,9.0,9.0,9.0,16.0
535,9.0,7.0,4.0,2.0,3.0
536,14.0,8.0,18.0,19.0,20.0


In [131]:
model = linear_model.LinearRegression()
model.fit(train_X, train_y)

LinearRegression()

In [132]:
# params = {}
# best_model = GridSearchCV(model, param_grid = params).fit(train_X, train_y)

In [133]:
pd.concat([pd.DataFrame(train_X.columns),pd.DataFrame(np.transpose(model.coef_))], axis = 1)


,0,0
0,LapPosition_1,0.028152
1,LapPosition_2,0.030044
2,PrevPosition1,0.311296
3,PrevPosition2,0.295320
4,PrevPosition3,0.215498


In [134]:
test_X = test_data[X_cols]
test_y = test_data['Position']

In [135]:
pred = model.predict(test_X)

In [136]:
r2_score(train_data['Position'], train_data['PrevPosition1'])

0.33950853078486987

In [137]:
results = pd.DataFrame({'pred': pred, 'actual': test_y})

In [138]:
results

,pred,actual
543,6.849229,3.0
544,16.675994,20.0
545,10.551196,17.0
546,12.823331,13.0
548,13.003049,9.0
...,...,...
888,14.124330,20.0
889,12.164450,2.0
891,4.426009,8.0
892,13.238270,15.0


In [139]:
results.describe()

,pred,actual
count,297.000000,297.000000
mean,10.132017,10.158249
std,4.035445,5.707263
min,1.976782,1.000000
25%,6.391572,5.000000
50%,11.182190,10.000000
75%,13.443080,15.000000
max,17.946123,20.000000


In [141]:
mean_squared_error(results['pred'], results['actual'])

16.14282628369124

In [140]:
joblib.dump(model, 'model.pkl')

['model.pkl']